In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

# improt anova and tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/normalized_feature_selected_output.parquet"
)
df = pd.read_parquet(file_path)
df.head()

,Metadata_ImageNumber,Metadata_Image_FileName_OP,Metadata_ObjectNumber,Metadata_ConvertImageToObjects_Number_Object_Number,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxArea,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_Y,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_Y,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,1,MAX_high_10_L.tiff,1,1,38250.0,269.0,182.0,44.0,12.0,131.546149,...,0.841475,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617
1,2,MAX_high_10_R.tiff,1,1,34170.0,208.0,245.0,38.0,44.0,106.962058,...,0.452480,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922
2,3,MAX_high_11_L.tiff,1,1,41736.0,250.0,267.0,62.0,45.0,131.359827,...,0.494813,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763
3,4,MAX_high_11_R.tiff,1,1,43616.0,212.0,272.0,24.0,40.0,101.069901,...,0.878729,0.865617,1.120905,1.140604,1.074529,1.117941,1.031585,1.015172,1.035150,1.019188
4,5,MAX_high_12_L.tiff,2,2,25894.0,283.0,155.0,69.0,34.0,164.579054,...,0.476768,0.482674,0.177886,0.147933,0.103313,0.156457,0.111765,0.107846,0.117614,0.108859


In [3]:
# combine the genotype and idenity columns
df["Metadata_unique"] = df["Metadata_genotype"] + "_" + df["Metadata_identity"]
# split the features and the metadata
metadata = df.columns.str.contains("Metadata")
# filter the metadata
metadata_df = df.loc[:, metadata]
# filter the features
features_df = df.loc[:, ~metadata]
features_df["Metadata_unique"] = metadata_df["Metadata_unique"]
features_df

/tmp/ipykernel_406442/508609000.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df['Metadata_unique'] = metadata_df['Metadata_unique']


,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,AreaShape_CentralMoment_1_3,AreaShape_CentralMoment_2_0,...,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256,Metadata_unique
0,1.706234,1.706234,2.652373,3.280425,-1.992966,0.184630,-1.404376,1.504670,-0.854028,0.297514,...,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617,high_both
1,0.771674,0.771674,-1.332747,0.164304,0.244371,0.173760,-1.885144,0.145870,-0.823045,0.950959,...,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922,high_both
2,2.180858,2.180858,2.142686,1.838038,-0.077515,0.008650,0.876597,-0.139787,1.467120,2.535223,...,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763,high_both
3,2.081557,2.081557,-0.880954,1.014901,0.660697,3.593763,-1.417922,-0.188826,-1.625716,3.032180,...,0.865617,1.120905,1.140604,1.074529,1.117941,1.031585,1.015172,1.035150,1.019188,high_both
4,0.659368,0.659368,-0.472511,1.810874,-1.929153,0.242599,-0.841314,0.664103,-0.831532,-0.461882,...,0.482674,0.177886,0.147933,0.103313,0.156457,0.111765,0.107846,0.117614,0.108859,high_both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.057291,0.057291,0.117036,-0.275397,0.333077,1.209451,1.466370,0.906241,0.931120,-0.029690,...,0.874221,1.067855,1.180034,1.008188,0.963815,1.013337,0.989966,1.021810,1.040963,wt_op
132,-1.123608,-1.123608,0.077732,-0.830730,0.036418,0.055958,0.178429,-0.201055,0.293556,-0.740213,...,-2.859015,-1.054336,-1.036668,-1.044983,-1.049984,-1.052059,-1.053833,-1.055638,-1.052029,wt_op
133,-0.902056,-0.902056,0.115831,-0.653438,0.005760,0.046615,0.218151,-0.181101,0.275091,-0.833798,...,-1.260327,-0.980259,-0.971405,-0.980991,-0.982947,-0.982787,-0.985621,-0.985673,-0.983364,wt_br
134,-0.742165,-0.742165,0.249972,-0.813136,0.045546,0.188770,0.663831,-0.144405,0.339618,-0.305887,...,-1.287340,-0.962709,-0.936314,-0.941929,-0.965431,-0.952584,-0.956938,-0.955804,-0.950883,wt_br


In [4]:
# anova and tukeyhsd for each feature
# create a list to store the results
anova_results = pd.DataFrame()

# loop through each feature
for feature in features_df.columns[:-1]:
    # create a model
    model = ols(f"{feature} ~ C(Metadata_unique)", data=features_df).fit()
    # create an anova table
    anova_table = sm.stats.anova_lm(model, typ=2)
    # create a tukeyhsd table
    tukeyhsd = pairwise_tukeyhsd(features_df[feature], features_df["Metadata_unique"])

    # get the f-statistic based p-value
    anova_p_value = anova_table["PR(>F)"][0]
    tmp = pd.DataFrame(
        tukeyhsd._results_table.data, columns=tukeyhsd._results_table.data[0]
    ).drop(0)
    tmp.reset_index(inplace=True, drop=True)
    # drop the first row
    tmp["feature"] = feature
    tmp["anova_p_value"] = anova_p_value
    # tmp['unique'] = tmp['group1'] + "_" + tmp['group2'] + "_" + feature
    tmp = pd.DataFrame(tmp)

    anova_results = pd.concat([anova_results, tmp], axis=0).reset_index(drop=True)

In [5]:
anova_results

,group1,group2,meandiff,p-adj,lower,upper,reject,feature,anova_p_value
0,high_both,unsel_both,0.462,0.8397,-0.6587,1.5827,False,AreaShape_Area,1.655652e-35
1,high_both,unsel_br,-2.0857,-0.0,-2.507,-1.6644,True,AreaShape_Area,1.655652e-35
2,high_both,unsel_op,-1.249,0.0,-1.6703,-0.8276,True,AreaShape_Area,1.655652e-35
3,high_both,wt_br,-2.2954,-0.0,-2.7047,-1.8862,True,AreaShape_Area,1.655652e-35
4,high_both,wt_op,-1.7362,-0.0,-2.1454,-1.327,True,AreaShape_Area,1.655652e-35
...,...,...,...,...,...,...,...,...,...
3655,unsel_br,wt_br,0.7651,0.0402,0.0206,1.5095,True,Texture_Variance_OP_3_03_256,2.678518e-04
3656,unsel_br,wt_op,1.2928,0.0,0.5484,2.0372,True,Texture_Variance_OP_3_03_256,2.678518e-04
3657,unsel_op,wt_br,0.0823,0.9995,-0.6621,0.8268,False,Texture_Variance_OP_3_03_256,2.678518e-04
3658,unsel_op,wt_op,0.6101,0.1744,-0.1344,1.3545,False,Texture_Variance_OP_3_03_256,2.678518e-04


In [6]:
# export the anova results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
anova_results_paht = out_dir / "anova_results.parquet"
anova_results.to_parquet(anova_results_paht)